# Deutsch-Jozsa and Bernstein-Vazirani Algorithms Workbook

This workbook describes the solutions to the problems offered in the "Deutsch-Jozsa and Bernstein-Vazirani Algorithms" kata. Since the tasks are offered as programming problems, the explanations also cover some elements of Qiskit that might be non-obvious for a first-time user.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister

## Exercise 1. Oracle for f(x) = f(x) = most significant bit of x

The binary representation of $x$ is $x = (x_{0}, x_{1}, \dots, x_{N-1})$, with the most significant bit encoded in the last bit (stored in the last qubit of the input array). Then, you can rewrite the function as

$$f(x) = x_{N-1}$$

and its effect on the quantum state as 

$$U_f \ket{x} = (-1)^{f(x)} \ket{x} = \ket{x_{0} } \otimes \cdots \otimes \ket{x_{N-2} } \otimes (-1)^{x_{N-1}}  \ket{x_{N-1}}$$

As you've seen in the demo, this can be achieved by applying a $Z$ gate to the last qubit.

In [ ]:
def oracle_msb_x(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.z(qr[-1])

## Exercise 2. Oracle for f(x) = parity of the number of 1 bits in x

In this oracle the answer depends on all bits of the input. You can write $f(x)$ as follows (here $\bigoplus$ denotes sum modulo $2$):

$$f(x) = \bigoplus_{k=0}^{N-1} x_k$$

Let's substitute this expression in the expression for the oracle effect on the quantum state:

$$U_f \ket{x} = (-1)^{f(x)} \ket{x} = (-1)^{\bigoplus_{k=0}^{N-1} x_k} \ket{x}$$

Since $(-1)^2 = 1$, you can replace sum modulo $2$ with a regular sum in the exponent. Then you'll be able to rewrite it as a product of individual exponents for each bit:

$$U_f \ket{x} = (-1)^{\sum_{k=0}^{N-1} x_k} \ket{x} = \prod_{k=0}^{N-1} {(-1)^{x_k}} \ket{x}$$

Now let's spell out the system state as a tensor product of individual qubit states:

$$U_f \ket{x} = \prod_{k=0}^{N-1} {(-1)^{x_k}} \cdot \ket{x_{0} } \otimes \cdots \otimes \ket{x_{N-1}}$$

Tensor product is a linear operation, so you can bring each $(-1)^{x_k}$ scalar factor in next to the corresponding $\ket{x_k}$:

$$U_f \ket{x} = (-1)^{x_0} \ket{x_{k}} \otimes \dots \otimes (-1)^{x_{N-1}} \ket{x_{N-1}} = \bigotimes_{k=0}^{N-1} (-1)^{x_k} \ket{x_{k}}$$

As you've seen in the previous oracle, this can be achieved by applying a $Z$ gate to each qubit.

In [ ]:
def oracle_parity(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.z(qr)

## Exercise 3. Implement Deutsch-Jozsa algorithm

Follow the algorithm as outlined in the previous section:

1. Create a `QuantumRegister` and a `ClassicalRegister` of length $N$, and create a `QuantumCircuit` with them. The qubits start in the $\ket{0}$ state.
2. Apply the $H$ gate to each qubit. You can use `ApplyToEach` operation for this, or a `for` loop.
3. Apply the oracle. The syntax for applying the oracle is the same as for applying any other Python function; remember to pass both `QuantumCircuit` and `QuantumRegister` variables as arguments.
4. Apply the $H$ gate to each qubit again.
5. Measure the qubits. 
6. Create a simulator and run the circuit using it. Since the algorithm is deterministic, you need just one shot. If the measurement result is `0...0`, the function is constant, and you need to return `True`, otherwise it's variable, and you need to return `False`.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator

def is_function_constant(n: int, oracle: callable) -> bool:
    # Define the quantum circuit
    qr = QuantumRegister(n)
    cr = ClassicalRegister(n)
    circ = QuantumCircuit(qr, cr)

    # Apply the Hadamard gates
    circ.h(qr)

    # Apply the oracle
    oracle(circ, qr)

    # Apply the Hadamard gates
    circ.h(qr)

    # Measure
    circ.measure(qr, cr)

    # Run the simulation and get the results
    simulator = AerSimulator(method='statevector')
    res_counts = simulator.run(circ, shots=1).result().get_counts()
    res_str = list(res_counts.keys())[0]
    return res_str == ('0' * n)

## Exercise 4. Implement Bernstein-Vazirani algorithm

The algorithm implementation is very similar to that of Deutsch-Jozsa algorithm, except the last step on which you analyze the measurement results.

1. Create a `QuantumRegister` and a `ClassicalRegister` of length $N$, and create a `QuantumCircuit` with them. The qubits start in the $\ket{0}$ state.
2. Apply the $H$ gate to each qubit. You can use `ApplyToEach` operation for this, or a `for` loop.
3. Apply the oracle. The syntax for applying the oracle is the same as for applying any other Python function; remember to pass both `QuantumCircuit` and `QuantumRegister` variables as arguments.
4. Apply the $H$ gate to each qubit again.
5. Measure the qubits. 
6. Create a simulator and run the circuit using it. The answer is the measurement results for each qubit, converted to bits $0$ and $1$. Remember that Qiskit reverses measurement results compared to the order of qubits, so you need to reverse them too.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator

def bernstein_vazirani_algorithm(n: int, oracle: callable) -> list[int]:
    # Define the quantum circuit
    qr = QuantumRegister(n)
    cr = ClassicalRegister(n)
    circ = QuantumCircuit(qr, cr)

    # Apply the Hadamard gates
    circ.h(qr)

    # Apply the oracle
    oracle(circ, qr)

    # Apply the Hadamard gates
    circ.h(qr)

    # Measure
    circ.measure(qr, cr)

    # Run the simulation and get the results
    simulator = AerSimulator(method='statevector')
    res_counts = simulator.run(circ, shots=1).result().get_counts()
    res_str = (list(res_counts.keys())[0])[::-1]
    return [1 if c == '1' else 0 for c in res_str]